# 04. Deploying Dask

## Overview

In this notebook, you will learn how to:

 - Configure remote Dask.distributed deployment.
 - Deploy Dask.distributed scheduler and workers on compute nodes.
 - Access scheduler and worker dashboards.

## Import idact

It's recommended that *idact* is installed with *pip*.  
Alternatively, make sure the dependencies are installed: `pip install -r requirements.txt`, and add *idact* to path, for example:  
`import sys`  
`sys.path.append('<YOUR_IDACT_PATH>')`

We will use a wildcard import for convenience:

In [ ]:
from idact import *
import bitmath

## Load the cluster

Let's load the environment and the cluster. Make sure to use your cluster name.

In [ ]:
load_environment()
cluster = show_cluster("test")
cluster

In [ ]:
access_node = cluster.get_access_node()
access_node.connect()

## Configure remote Dask deployment

### Install Dask.distributed on the cluster

Make sure `dask`, `distributed` and `bokeh` packages are installed with the Python 3.5+ distribution you intend to use on the cluster, see [Dask documentation](http://distributed.dask.org/en/latest/install.html).

If you encounter any problems with deployment, this may be due to some library versions being incompatible. You can try installing frozen versions included with the *idact* repo in `envs/dask_jupyter_tornado.txt`, same as described in the previous tutorial `03. Deploying Jupyter`.

### Specify setup actions

Same as for the Jupyter deployment, in order for *idact* to find and execute the proper binaries, you'll need to specify setup steps as a list of Bash script lines.

They may very well be the exact same instructions as for the Jupyter deployment. If they are not, replace `cluster.config.setup_actions.jupyter` with the correct instructions list.

In [ ]:
cluster.config.setup_actions.dask = cluster.config.setup_actions.jupyter
save_environment()

### Choose the scratch directory

Dask requires a directory for offloading data when the memory starts to fill up. Usually, the faster the storage, the better, see `--local-directory` in [dask-worker documentation](http://distributed.dask.org/en/latest/worker.html#spill-data-to-disk).

You can pass an absolute path, or a cluster environment variable.

In [ ]:
cluster.config.scratch = '$SCRATCH'
save_environment()

## Allocate nodes for the Dask deployment

We will deploy Dask on three nodes: a scheduler on the first node, and one worker on each node. Make sure to adjust the `--account` parameter, same as in previous notebooks.

In [ ]:
nodes = cluster.allocate_nodes(nodes=3,
                               cores=2,
                               memory_per_node=bitmath.GiB(10),
                               walltime=Walltime(minutes=10),
                               native_args={
                                   '--account': 'intdata'
                               })
nodes

In [ ]:
nodes.wait()
nodes

## Deploy Dask

After the initial setup, Dask can be deployed with a single command:

In [ ]:
dd = deploy_dask(nodes)
dd

If the deployment fails, take a look at `idact.log` to find out why.

You can get a Dask client for the deployment:

In [ ]:
client = dd.get_client()
client

You shouldn't perform computations with Dask.distributed from your local computer, due to likely Python and library version mismatches.

Even if your Python environment matched the cluster exactly, the amount of data that could be transferred to your local computer could prove overwhelming.

We will address this issue in one of the next notebooks, by making the Dask deployment accessible from a notebook deployed on the cluster.

## Browse Dask dashboards

Dask provides dashboards for the scheduler and each worker:

In [ ]:
dd.diagnostics.addresses

To open all dashboards, execute the line below. You can also click the scheduler dashboard link under `get_client` above.

In [ ]:
dd.diagnostics.open_all()

## Cancel Dask deployment

After you're done, you can cancel the deployment by calling `cancel`, though it will be killed anyway when the node allocation ends.

In [ ]:
dd.cancel()

Alternatively, the following will just close the tunnels, without attempting to kill Dask scheduler and workers:

In [ ]:
dd.cancel_local()

Dask client is multithreaded, so it needs to be closed.

In [ ]:
client.close()

## Cancel the allocation

It's important to cancel an allocation if you're done with it early, in order to minimize the CPU time you are charged for.

In [ ]:
nodes.running()

In [ ]:
nodes.cancel()

In [ ]:
nodes.running()

## Next notebook

In the next notebook, we will install *idact* on the cluster and configure it from a deployed Jupyter Notebook.